In [1]:
# packages for data manipulation

import numpy as np
import pandas as pd

# packages for visualisation
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Read in sales data
sales = pd.read_csv('dummy_data.csv', encoding='utf-8')

In [4]:
with open('sales_data_sample.csv') as f:
    print(f)

<_io.TextIOWrapper name='sales_data_sample.csv' mode='r' encoding='UTF-8'>


In [5]:
sales.head()

,STATE,STORE,WEEK,SESSION,PROPOSITION,CUST_ID,ACTIVE_SUB,ACTUAL_SALES,FORECAST_SALES,ACTUAL_ORDERS,FORECAST_ORDERS
0,NSW,1,1,Morning,pick-up,50,0,176,180,44,43
1,NSW,1,1,Morning,delivery,28,1,500,515,100,105
2,NSW,1,1,Morning,in-store,16,0,450,450,90,95
3,NSW,1,1,Afternoon,pick-up,33,1,465,510,93,96
4,NSW,1,1,Afternoon,delivery,21,0,176,126,44,44


In [6]:
# Add column with weekly sales
weekly_sales = sales.groupby('WEEK', as_index=False).ACTUAL_SALES.sum()
weekly_sales.rename(columns={'ACTUAL_SALES':'WEEKLY_SALES'}, inplace=True)
weekly_sales['WEEKLY_DIFF'] = weekly_sales['WEEKLY_SALES'].diff().fillna(0)
weekly_sales.head()

,WEEK,WEEKLY_SALES,WEEKLY_DIFF
0,1,6572,0.0
1,2,6588,16.0
2,3,5976,-612.0
3,4,5662,-314.0


In [7]:
weekly_sales['WEEK_ON_WEEK'] = (weekly_sales['WEEKLY_SALES'] + weekly_sales['WEEKLY_DIFF'])/weekly_sales['WEEKLY_SALES']*100
weekly_sales.head()

,WEEK,WEEKLY_SALES,WEEKLY_DIFF,WEEK_ON_WEEK
0,1,6572,0.0,100.000000
1,2,6588,16.0,100.242866
2,3,5976,-612.0,89.759036
3,4,5662,-314.0,94.454256


In [8]:
# SORT DATAFRAME BY WEEK AND BY PROPOSITION
sales = sales.sort_values(['WEEK', 'PROPOSITION'])

In [9]:
weekly_orders = sales.groupby('WEEK', as_index=False).count()
weekly_orders = weekly_orders[['WEEK', 'ACTUAL_ORDERS']]
weekly_orders.rename(columns={'ACTUAL_ORDERS':'WEEKLY_ORDERS'}, inplace=True)
weekly_orders.head() 

,WEEK,WEEKLY_ORDERS
0,1,27
1,2,27
2,3,27
3,4,27


In [33]:
# Non-variable function to compute last week's sales vs previous week's sales
def week_on_week():
    
    # Add column with weekly sales
    weekly_sales = sales.groupby('WEEK', as_index=False).ACTUAL_SALES.sum()
    weekly_sales.rename(columns={'ACTUAL_SALES':'WEEKLY_SALES'}, inplace=True)
    weekly_sales['WEEKLY_DIFF'] = weekly_sales['WEEKLY_SALES'].diff().fillna(0)
    
    # Compute week on week (percentage)
    weekly_sales['WEEK_ON_WEEK'] = (weekly_sales['WEEKLY_SALES'] + weekly_sales['WEEKLY_DIFF'])/weekly_sales['WEEKLY_SALES']*100
    
    return weekly_sales

In [46]:
# Variable version of the function to compute last 'n' weeks' sales vs previous 'n' weeks' sales
def var_week_on_week(n):
    
    # Add column with weekly sales
    weekly_sales = sales.groupby('WEEK', as_index=False).ACTUAL_SALES.sum()
    weekly_sales.rename(columns={'ACTUAL_SALES':'WEEKLY_SALES'}, inplace=True)
    
    # Add table aggregating the last 'n' weeks' sales
    n_weekly_sales = weekly_sales.groupby(weekly_sales.index // n).sum()
    n_weekly_sales.insert(0, 'BLOCK_NUMBER', range(1, 1 + len(n_weekly_sales)))
    n_weekly_sales = n_weekly_sales[['BLOCK_NUMBER', 'WEEKLY_SALES']]
    n_weekly_sales.rename(columns={'WEEKLY_SALES':'BLOCK_SALES'}, inplace=True)
    n_weekly_sales['BLOCK_DIFF'] = n_weekly_sales['BLOCK_SALES'].diff().fillna(0)
    
    # Compute block on block (percentage)
    n_weekly_sales['BLOCK_ON_BLOCK'] = (n_weekly_sales['BLOCK_SALES'] + n_weekly_sales['BLOCK_DIFF'])/n_weekly_sales['BLOCK_SALES']*100
    return n_weekly_sales

In [47]:
week_on_week()

,WEEK,WEEKLY_SALES,WEEKLY_DIFF
0,1,6572,0.0
1,2,6588,16.0
2,3,5976,-612.0
3,4,5662,-314.0


In [48]:
var_week_on_week(1)

,BLOCK_NUMBER,BLOCK_SALES,BLOCK_DIFF,BLOCK_ON_BLOCK
0,1,6572,0.0,100.000000
1,2,6588,16.0,100.242866
2,3,5976,-612.0,89.759036
3,4,5662,-314.0,94.454256


In [49]:
var_week_on_week(2)

,BLOCK_NUMBER,BLOCK_SALES,BLOCK_DIFF,BLOCK_ON_BLOCK
0,1,13160,0.0,100.000000
1,2,11638,-1522.0,86.922152


In [50]:
var_week_on_week(3)

,BLOCK_NUMBER,BLOCK_SALES,BLOCK_DIFF,BLOCK_ON_BLOCK
0,1,19136,0.0,100.000000
1,2,5662,-13474.0,-137.972448
